<a href="https://colab.research.google.com/github/danielahernandez29/Curso-IA/blob/main/RNN_/Bitcoin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Se carga el archivo CSV
_df = pd.read_csv("/content/Bitcoin_1_1_2024-6_9_2024_historical_data_coinmarketcap.csv", delimiter=';')
# Lo ordeno
_df = _df.sort_index(ascending=False)
_df.head()

from sklearn.preprocessing import MinMaxScaler

# Escalo los datos para que estén entre 0 y 1
scaler = MinMaxScaler(feature_range=(0, 1))
# Se normalizan los datos
scaled_data = scaler.fit_transform(dates)

# Cuantos datos se usan para predecir
window_size = 60

import numpy as np

# Se entrena al modelo
# Básicamente agarra 60 valores y usa el que sigue como etiqueta

def create_sequences(data, window_size):
    sequences = []
    labels = []
    for i in range(len(data) - window_size):
        sequences.append(data[i:i+window_size])
        labels.append(data[i + window_size, 0])  # Lo que quiero predecir
    return np.array(sequences), np.array(labels)

# Genero X (las secuencias) y Y (la predicción real)
X, y = create_sequences(scaled_data, window_size)

# Divido mis datos: entrenamiento y prueba
split = int(len(X) * 0.8)
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, SimpleRNN

# Armo el modelo RNN
model = Sequential()
# Primera capa RNN
model.add(SimpleRNN(units=120, return_sequences=True, input_shape=(window_size, X_train.shape[2])))

model.add(SimpleRNN(units=60, return_sequences=False))

model.add(Dense(units=30))
# Última capa da un solo valor
model.add(Dense(units=1))

from tensorflow.keras.optimizers import Adam

# velocidad de aprendizaje
learning_rate = 0.001
adam_optimizer = Adam(learning_rate=learning_rate)

# se compila el modelo
model.compile(optimizer=adam_optimizer, loss='mean_squared_error')

# Entreno el modelo
model.fit(X_train, y_train, batch_size=1, epochs=10)

# Hago predicciones
predictions = model.predict(X_test)

predictions = scaler.inverse_transform(np.concatenate((predictions, np.zeros((predictions.shape[0], 3))), axis=1))[:,0]

# Desescalo los valores reales
y_test = scaler.inverse_transform(np.concatenate((y_test.reshape(-1, 1), np.zeros((y_test.shape[0], 3))), axis=1))[:,0]

from sklearn.metrics import mean_absolute_error, mean_squared_error

# Calculo las métricas para ver que tan bien predijo
mae = mean_absolute_error(y_test, predictions)
rmse = np.sqrt(mean_squared_error(y_test, predictions))

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")

import matplotlib.pyplot as plt
import numpy as np

valid = df[split:]

valid = valid.reset_index(drop=True)
# Creo columna donde voy a poner las predicciones
valid['Predictions'] = np.nan

# Meto las predicciones
valid.loc[window_size:, 'Predictions'] = predictions

dates_valid = pd.to_datetime(valid['timeOpen']).apply(lambda x: x.strftime('%Y-%m-%d')).tolist()

# Grafico lo real vs lo predicho
plt.figure(figsize=(16,8))
plt.title('Modelo RNN para Predicción de Bitcoin')
plt.xlabel('Fecha')
plt.ylabel('Precio de Bitcoin (USD)')
plt.plot(dates_valid, valid[['close', 'Predictions']])
plt.legend(['Valor Real', 'Predicciones'], loc='lower right')
plt.xticks(rotation=90)
plt.show()


# Aqui guardo las secuencias futuras
future_sequences = []
# Uso la ultima secuencia disponible
last_sequence = X[-1]

# Cuantos dias quiero predecir
days = 10
for _ in range(days):
    # Predigo el siguiente valor con el modelo
    next_value = model.predict(np.array([last_sequence]))[0, 0]

    # Actualizo la secuencia
    last_sequence = np.concatenate((last_sequence[1:], [[next_value]]), axis=0)

    # Guardo la secuencia
    future_sequences.append(last_sequence)

# Lo paso a numpy y doy forma para RNN
future_sequences = np.array(future_sequences)
future_sequences = np.reshape(future_sequences, (future_sequences.shape[0], future_sequences.shape[1], 1))

# Predigo los valores futuros
future_predictions = model.predict(future_sequences)

future_predictions = scaler.inverse_transform(np.concatenate((future_predictions, np.zeros((future_predictions.shape[0], 3))), axis=1))[:,0]

# Obtengo la ultima fecha real del dataset
last_date = df['timeOpen'].iloc[-1]

# Genero las fechas de los dias que estoy prediciendo
future_dates = pd.date_range(start=last_date, periods=days)[1:]
future_dates = future_dates.strftime('%Y-%m-%d').tolist()

# se grafica las predicciones futuras
plt.figure(figsize=(16,8))
plt.title('Predicciones de los siguientes días')
plt.xlabel('Fecha')
plt.ylabel('Precio de Bitcoin (USD)')
plt.plot(dates_valid, valid[['close', 'Predictions']], label=['real', 'Predicciones'])
plt.plot(future_dates, future_predictions[:-1], label='Predicciones')
plt.legend()
plt.xticks(rotation=90)
plt.show()
